#Problème - Session n°2 : une variable cachée

Dans ce problème, on travaille sur un jeu de données comportant 50.000 entrées $x_i$ et des cibles $y_i$. Les entrées sont des vecteurs de taille 10 (au format torch), les cibles sont des scalaires construits à partir de cinq fonctions différentes ($f_0$, ..., $f_4$) : \

$$ \forall i, \exists k\in [\![0 \;;4]\!]  \:\: \text{tel que} \: f_k(x_i) = y_i $$

Ces fonctions sont inconnues, ainsi que l'indice $k$. Par contre, on sait que le groupe des 1000 premières cibles ont été construites à partir du même indice  $k$, de même pour les mille  suivantes, et ainsi de suite.

Le but est de parvenir à rassembler les groupes de cibles qui ont été générées avec le même indice $k$ (avec la même fonction).

In [4]:
# Example d'échantillonnage du dataset
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

! git clone https://github.com/messagerjulien/exam_2025_session2.git
! cp exam_2025_session2/utils/utils.py .
from utils import Problem1Dataset

dataset = Problem1Dataset()
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

for batch in dataloader:
    x_batch, y_batch, k_batch, idx_batch = batch
    print("Batch input shape:", x_batch.shape)
    print("Batch target shape:", y_batch.shape)
    print("Batch k shape:", k_batch.shape) # indice k (pas utilisable à l'entraînement)
    print("Batch indices shape:", idx_batch.shape)
    break

fatal: destination path 'exam_2025_session2' already exists and is not an empty directory.
Batch input shape: torch.Size([32, 10])
Batch target shape: torch.Size([32, 1])
Batch k shape: torch.Size([32])
Batch indices shape: torch.Size([32])


**Consignes :**
- Entraîner l'architecture proposée dans la cellule suivante.
- Montrer que les vecteurs 2D de self.theta permettent de répondre
  au problème posé.
- Décrire le rôle de self.theta, du vector noise \
 et ainsi que la raison de la division par 1000 (**indices // 1000** dans le code).

In [6]:
class DeepMLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=256):
        super(DeepMLP, self).__init__()
        self.theta = nn.Parameter(torch.randn(50, 2))
        self.fc1 = nn.Linear(input_dim + 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x, indices):
        theta_batch = self.theta[indices // 1000, :]
        noise = torch.normal(mean=torch.zeros_like(theta_batch),
                             std=torch.ones_like(theta_batch))
        x = torch.cat([x, theta_batch + noise], dim=1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return x, theta_batch

In [ ]:
num_epochs = 2000
batch_size = 50 # A régler
loss_fn = nn.CrossEntropyLoss() # Standard pour ce type de tâche

model = DeepMLP(10, 5)
optimizer = torch.optim.SGD(model.parameters(), lr=10**(-3))
for epoch in range(num_epochs):
    print("Epoch", epoch)
    # random traversal of the dataset
    for x, y, k, indice in dataloader:
        # zeroing gradients
        optimizer.zero_grad()
        # calculation of (p0, p1)

        output, theta = model(x, indice) #J'ai voulu passer sur GPU mais j'avais une erreur donc j'ai enlevé (".cuda()")
        # calculation of the error

        l = loss_fn(output, k)
        # calculation of gradients
        l.backward()
        # weight update
        optimizer.step()


Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19
Epoch 20


self.theta représente un tenseur de paramètres qui sont disponibles uniquement pour cette classe. Il est de taille 50x2 et on rajoute theta en plus d'un bruit aux données d'entrées.

Le noise est un bruit centré réduit de la même taille que le tenseur theta.

On divise par mille parce qu'on sait d'après l'énoncé que les entrées sont créés par la même fonction, par groupe de 1000. Ainsi on utilise les mêmes paramètres de theta pour un même groupe de vecteurs.